신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다. <BR>
torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다. PyTorch의 모든 모듈은 nn.Module의 하위 클래스입니다. <br>
신경망은 다른 모듈(계층;layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# 학습을 위한 장치 얻기

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# 클래스 정의하기

내가 만들려고 하는 신경망 모델을 nn.Module의 하위클래스로 정의하고, __init__ 에서 신경망 계층들을 초기화합니다. nn.Module을 상속 받은 모든 클래스는 forward 메소드에 입력 데이터에 대한 연산들을 구현

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)                 
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NeuralNetwork의 인스턴스를 생성하고 이를 device로 이동한 뒤, 구조(Structure)를 출력

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


model을 사용하기 위해 입력데이터를 전달. 이는 일부 백그라운드 연산들과 함께 모델의 forward를 실행! <br>
model.forward()를 직접 호출하면 안됨!! <br><br>

모델에 입력을 전달하여 호출하면 2차원 텐서를 반환. 2차원 텐서의 dim=0은 각 클래스에 대한 원시(raw) 예측값 10개가, dim = 1에는 각 출력의 개별 값들이 해당. 원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻음




In [5]:
X = torch.rand(1, 28, 28, device = device)
#! 이렇게 실행하는 구나!
logits = model(X)

pred_proba = nn.Softmax(dim=1)(logits)
y_pred = pred_proba.argmax(1)
print('Predicted class : {}'.format(y_pred))


Predicted class : tensor([2], device='cuda:0')


In [7]:
logits

tensor([[-0.0083,  0.0514,  0.0623, -0.1255, -0.0408,  0.0137,  0.0273,  0.0256,
         -0.1014,  0.0612]], device='cuda:0', grad_fn=<AddmmBackward0>)

# 모델 계층(Layer)

FashionMNIST 모델의 계층들을 살펴보자. 이를 설명하기 위해, 28x28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아보자

In [8]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


#  nn.Flatten

nn.Flatten 계층을 초기화하여 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 반환. (dim=0의 미니배치 차원은 유지) <br>
■ dim = 0이란? 아래 방향 (행 방향)

In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


# nn.Linear

선형 계층은 저장된 가중치(weight)와 편향(bias)을 사용하여 입력에 선형 변환(linear transformation을 적용하는 모듈)

In [10]:
layer1 = nn.Linear(in_features = 28*28, out_features= 20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


# nn.ReLU

비선형 활성화(activation)는 모델의 입력과 출력 사이에 복잡한 관계를 만듬. 비선형 활성화는 선형 변환 후에 적용되어 비선형성을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 도움

In [11]:
print('Before ReLU: {}\n\n'.format(hidden1))
hidden1 = nn.ReLU()(hidden1)
print('After ReLU: {}\n\n'.format(hidden1))

Before ReLU: tensor([[-1.2431e-02, -1.5965e-01,  6.9422e-01, -9.9884e-02, -9.0068e-02,
         -2.6082e-02,  8.9905e-01, -2.1646e-01,  4.3618e-01, -4.3603e-01,
         -1.2491e-01,  3.8924e-01,  1.9218e-01, -1.0233e-01, -7.9198e-02,
         -9.0541e-02, -1.1108e-01,  6.1446e-01,  1.6459e-01,  1.0444e-01],
        [ 6.3221e-01, -2.0196e-01,  4.2849e-01,  1.3530e-01,  2.2741e-01,
         -7.6426e-02,  3.8316e-01, -3.1954e-01,  5.4336e-01, -5.0896e-01,
         -3.4891e-01,  6.9647e-01, -3.5113e-02, -4.7922e-01, -2.7591e-02,
          6.0458e-02, -5.6146e-01,  3.3308e-01,  8.6559e-05,  3.9843e-01],
        [ 2.0940e-01, -5.4754e-02,  7.9397e-01, -2.2608e-01, -5.7247e-02,
          1.2324e-01,  4.8153e-01,  3.0743e-01,  4.4212e-01, -4.9560e-01,
         -3.7261e-01,  3.6917e-01,  7.4173e-02,  1.0443e-01, -3.6371e-01,
         -9.5264e-02, -4.4460e-01,  5.3836e-01,  3.6164e-01,  2.5553e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000e+00, 0.0000e+00, 6.9422e-01, 0.0

# nn.Sequential

nn.Sequential은 순서를 갖는 모듈의 컨테이너. 데이터는 정의된 것과 같은 순서로 모든 모듈들을 통해 전달. 순차 컨테이너를 사용하여 아래의 seq_modules와 같은 신경망을 빠르게 만들 수 있음

In [12]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)

input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

# nn.Softmax

In [17]:
softmax = nn.Softmax(dim = 0)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.3123, 0.3038, 0.3577, 0.3467, 0.3278, 0.3827, 0.3534, 0.3181, 0.3110,
         0.3272],
        [0.3474, 0.3519, 0.3074, 0.3295, 0.3525, 0.3256, 0.3337, 0.3730, 0.3579,
         0.3205],
        [0.3403, 0.3443, 0.3350, 0.3238, 0.3197, 0.2918, 0.3129, 0.3089, 0.3311,
         0.3523]], grad_fn=<SoftmaxBackward0>)


# 모델 매개변수

신경망 내부의 많은 계층들은 매개변수화(parameterize)됩니다. 즉, 학습 중에서 최적화되는 가중치와 편향과 연과되어집니다. <br>
nn.Module을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적되며, 모델의 parameters() 밑 named_parameters() 메소드로 모든 매개변수에 접근할 수 있음

In [18]:
print('Model Structure: {}\n\n'.format(model))

for name, param in model.named_parameters():
    print('Layer: {}, Size: {}, Values: {}\n'.format(name, param.size(), param[:2]))

Model Structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight, Size: torch.Size([512, 784]), Values: tensor([[-0.0294, -0.0170, -0.0107,  ...,  0.0145, -0.0033,  0.0174],
        [-0.0345,  0.0029, -0.0023,  ...,  0.0332,  0.0330, -0.0270]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias, Size: torch.Size([512]), Values: tensor([-0.0220, -0.0313], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight, Size: torch.Size([512, 512]), Values: tensor([[ 0.0195, -0.0065, -0.0343,  ...,  0.0406,  0.0337, -0.0030],
        [-0.0433, -0.0413, -0.0440,  ..., -0.0248,  0.0058, -0.0150]],
       device='cuda:0', grad_fn=<SliceBackward